In [38]:
import pandas as pd
import re
import numpy as np
from sklearn.preprocessing import OneHotEncoder

In [37]:
# data cleaning
def clean_data(df):
    
    # convert price $ to numeric
    df["price"] = df["price"].replace("[\$,]", "", regex=True).astype(float)
    
    # handle missing values
    df["bathrooms"] = df["bathrooms"].fillna(df["bathrooms"].median())
    df["bedrooms"] = df["bedrooms"].fillna(df["bedrooms"].median())
    df["beds"] = df["beds"].fillna(df["beds"].median())
    

    return df

# Clean the data
df_clean = clean_data(df)


In [40]:
#print(df_clean)
#df_clean["amenities"]
enc = OneHotEncoder(sparse_output=True, handle_unknown='ignore')
all_amenities = set(amenity for sublist in df_clean['amenities'] for amenity in sublist)
all_amenities_list = np.array(list(all_amenities)[0:4]).reshape(-1, 1)
enc.fit_transform(all_amenities_list)
# print(enc.get_feature_names_out())
#df_clean["amenities"]
enc.transform([["Hair dryer"]]).toarray()

array([[0., 0., 0., 0.]])